In [4]:
from datasets import load_dataset
import os

os.environ["http_proxy"] = "http://127.0.0.1:7895"
os.environ["https_proxy"] = "http://127.0.0.1:7895"


ds_test_task = load_dataset("osunlp/Multimodal-Mind2Web", split="test_task")
ds_test_website = load_dataset("osunlp/Multimodal-Mind2Web", split="test_website")
ds_test_domain = load_dataset("osunlp/Multimodal-Mind2Web", split="test_domain")
ds_train = load_dataset("osunlp/Multimodal-Mind2Web", split="train")

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [4]:
for key, value in ds_test_domain[2].items():
    if "html" not in key:
        print("{}: {}".format(key, value))

action_uid: bd937d20-eae1-4028-958b-be2d2a0dd9b8
operation: {"original_op": "TYPE", "value": "60505", "op": "TYPE"}
pos_candidates: ['{"tag": "input", "attributes": "{\\"backend_node_id\\": \\"6848\\", \\"bounding_box_rect\\": \\"70,331.1875,514,44\\", \\"type\\": \\"text\\", \\"id\\": \\"city-state-input\\", \\"class\\": \\"form-control location-search-input\\", \\"placeholder\\": \\"City and State, or ZIP Code\\", \\"input_value\\": \\"\\", \\"is_clickable\\": \\"true\\", \\"data_pw_testid_buckeye_candidate\\": \\"1\\"}", "is_original_target": true, "is_top_level_target": true, "backend_node_id": "6848"}']
neg_candidates: ['{"tag": "html", "attributes": "{\\"backend_node_id\\": \\"6853\\", \\"bounding_box_rect\\": \\"0,0,1280,1359.375\\", \\"class\\": \\" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow 

In [59]:
import numpy as np
from typing import List, Dict, Tuple
import json
import os
from openai import OpenAI
import faiss
from tqdm import tqdm


class ExemplarMemory:
    def __init__(self, model="text-embedding-ada-002", top_k=3):
        # 确保OpenAI API密钥已设置
        self.client = OpenAI(
            api_key="sk-VBtOoMbXC0U5ESUL9d2c39979bE548Cd9cDa8fE1F2264c83",
            base_url="https://aihubmix.com/v1",
        )
        self.model = model
        self.top_k = top_k

        # 存储空间
        self.dimension = 1536
        self.index = None  # FAISS索引
        self.data_index = []  # 存储对应的轨迹

    def encode_metadata(self, metadata: Dict) -> np.ndarray:
        """将任务元数据编码为向量"""
        # 构建输入文本
        text = f"website: {metadata['website']} domain: {metadata['domain']}subdomain: {metadata['subdomain']} task_description: {metadata['task']}"

        # 使用OpenAI API进行编码
        response = self.client.embeddings.create(model=self.model, input=text)
        embedding = np.array(response.data[0].embedding, dtype=np.float32)
        return embedding

    def init_memory(self, data):
        """从数据初始化记忆库"""
        # 初始化FAISS索引
        self.index = faiss.IndexFlatL2(self.dimension)  # 使用L2距离

        unique_annotation_ids = set()
        for i in tqdm(range(len(data)), desc="初始化记忆"):
            metadata = {}
            metadata["website"] = data[i]["website"]
            metadata["domain"] = data[i]["domain"]
            metadata["subdomain"] = data[i]["subdomain"]
            metadata["task"] = data[i]["confirmed_task"]
            annotation_id = data[i]["annotation_id"]
            if annotation_id not in unique_annotation_ids:
                unique_annotation_ids.add(annotation_id)
                data_index = i
                self.add_example(metadata, data_index)

    def add_example(self, metadata: Dict, index: int):
        """添加新的示例到内存"""
        embedding = self.encode_metadata(metadata)

        # 如果索引尚未初始化，则初始化它
        if self.index is None:
            self.dimension = embedding.shape[0]
            self.index = faiss.IndexFlatL2(self.dimension)

        # 将向量添加到FAISS索引
        self.index.add(embedding.reshape(1, -1))
        self.data_index.append(index)

    def save_memory(self, path):
        """保存记忆库到磁盘"""
        os.makedirs(path, exist_ok=True)

        # 保存FAISS索引
        faiss.write_index(self.index, os.path.join(path, "memory.index"))

        # 保存轨迹
        with open(os.path.join(path, "trajectories.json"), "w") as f:
            json.dump(self.trajectories, f, ensure_ascii=False, indent=2)

    def load_memory(self, path):
        """从磁盘加载记忆库"""
        # 加载FAISS索引
        self.index = faiss.read_index(os.path.join(path, "memory.index"))

        # 加载轨迹
        with open(os.path.join(path, "trajectories.json"), "r") as f:
            self.trajectories = json.load(f)

    def query(self, query_metadata: Dict) -> List[Tuple[List, float]]:
        """查询最相似的轨迹"""
        # 编码查询
        query_embedding = self.encode_metadata(query_metadata)

        # 使用FAISS进行相似度搜索
        D, I = self.index.search(query_embedding.reshape(1, -1), self.top_k)

        # 获取结果
        results = []
        for i in range(min(self.top_k, len(I[0]))):
            idx = I[0][i]
            if idx >= 0 and idx < len(self.data_index):  # 确保索引有效
                similarity = 1 / (1 + D[0][i])  # 转换距离为相似度
                results.append((self.data_index[idx], similarity))

        return results

    def save_memory(self, path="/home/zyy/Mind2Web/similarity_data"):
        """保存记忆库到磁盘"""
        os.makedirs(path, exist_ok=True)

        # 保存FAISS索引
        faiss.write_index(self.index, os.path.join(path, "memory.index"))

        # 保存轨迹
        with open(os.path.join(path, "data_index.json"), "w") as f:
            json.dump(self.data_index, f, ensure_ascii=False, indent=2)

    def load_memory(self, path="/home/zyy/Mind2Web/similarity_data"):
        """从磁盘加载记忆库"""
        # 加载FAISS索引
        self.index = faiss.read_index(os.path.join(path, "memory.index"))

        # 加载轨迹
        with open(os.path.join(path, "data_index.json"), "r") as f:
            self.data_index = json.load(f)


def get_metadata(data):
    metadata = {}
    metadata["website"] = data["website"]
    metadata["domain"] = data["domain"]
    metadata["task"] = data["confirmed_task"]
    metadata["subdomain"] = data["subdomain"]
    return metadata

In [60]:
memory = ExemplarMemory()
memory.init_memory(ds_train)
memory.save_memory()

初始化记忆:  33%|███▎      | 2578/7775 [08:54<19:41,  4.40it/s]  /home/zyy/anaconda3/envs/EasyRL/lib/python3.12/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (108679428 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
初始化记忆: 100%|██████████| 7775/7775 [26:14<00:00,  4.94it/s]  


In [64]:
query = get_metadata(ds_test_domain[0])

results = memory.query(query)

# 打印结果
print("query task:", query["task"])
print("query website:", query["website"])
print("query domain:", query["domain"])
print("query subdomain:", query["subdomain"])
print("query trajectory:", ds_test_domain[0]["action_reprs"])
print("\nsimilar task:")
for index, similarity in results:
    print(f"similarity: {similarity:.3f}")
    print("task:", ds_train[index]["confirmed_task"])
    print("website:", ds_train[index]["website"])
    print("domain:", ds_train[index]["domain"])
    print("subdomain:", ds_train[index]["subdomain"])
    print("trajectory:", ds_train[index]["action_reprs"])

query task: Book an appointment for applying new passport for one adult, Ellen Walker, with phone number 123-456-7890 and email address EW@gmail.com on April 4, 2023 at 1 pm in the post office nearest to zip code 60505. Don't send updates via text message.
query website: usps
query domain: Service
query subdomain: Shipping
query trajectory: ['[menuitem]  Quick Tools -> CLICK', '[p]  Find USPS Locations -> CLICK', '[textbox]  City and State, or ZIP Code™ -> TYPE: 60505', '[link]  Passport Appointments -> CLICK', '[link]  Schedule an Appointment -> CLICK', '[combobox]  Service Type -> SELECT: New Passport Only', '[combobox]  Adult(s) -> SELECT: 1', '[span]   -> CLICK', '[textbox]  *City and State or ZIP Code™ -> TYPE: 60505', '[textbox]  *City and State or ZIP Code™ -> CLICK', '[button]  Search -> CLICK', '[button]  Select Location -> CLICK', '[textbox]  *First Name -> TYPE: Ellen', '[textbox]  *Last Name -> TYPE: Walker', '[textbox]  *Phone Number -> TYPE: 123-456-7890', '[textbox]  *Em